# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [12]:
cities.columns

Index(['City_ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat',
       'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]

# * Narrow down the DataFrame to find your ideal weather condition. For example:
#  * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.

narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0) & (cities["Wind Speed"] > 10), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
2,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Pisco
1,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Koumac
2,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,Namibe


In [16]:
wind_speed = narrowed_city_df["Wind Speed"]
wind_speed

0    11.41
1    12.86
2    17.40
Name: Wind Speed, dtype: float64

In [17]:
hotel_loc_df = narrowed_city_df[["Hotel Name","City","Lat","Lng","Wind Speed","Country"]]
hotel_loc_df

,Hotel Name,City,Lat,Lng,Wind Speed,Country
0,Pisco,pisco,-13.71,-76.20,11.41,PE
1,Koumac,koumac,-20.56,164.28,12.86,NC
2,Namibe,namibe,-15.19,12.15,17.40,AO


In [18]:
hotel_loc_df.columns

Index(['Hotel Name', 'City', 'Lat', 'Lng', 'Wind Speed', 'Country'], dtype='object')

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#narrowed_city_df = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_lat_lon = hotel_loc_df[["Lat","Lng"]]
hotel_lat_lon.head()

,Lat,Lng
0,-13.71,-76.20
1,-20.56,164.28
2,-15.19,12.15


In [20]:
# Add marker layer ontop of heat map

# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(hotel_lat_lon, weights=wind_speed, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))